In [1]:
import os
os.chdir("../")

In [2]:

from dataclasses import dataclass
from pathlib import Path
import numpy as np

@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    base_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_image_size: list
    params_learning_rate: float

In [3]:
from pixel_predictor.constants import *
from pixel_predictor.utils.common import read_yaml,create_directories


In [4]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_training_config(self) -> TrainingConfig:
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model
        params = self.params
        training_data = self.config.training.training_data_path
        create_directories([
            Path(training.root_dir)
        ])

        training_config = TrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            base_model_path=Path(prepare_base_model.base_model_path),
            training_data=Path(training_data),
            params_learning_rate= self.params.LEARNING_RATE,
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_image_size=params.IMAGE_SIZE
        )

        return training_config

In [5]:
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
import numpy as np

In [6]:
class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config

    
    def get_base_model(self):
        self.model = tf.keras.models.load_model(
            self.config.base_model_path
        )
    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)
    def train(self):
        data = np.load(self.config.training_data)
        X, y = data['X'], data['y']
        X = X.reshape((X.shape[0], 50, 50, 1))
        X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
        self.model.compile(optimizer=Adam(learning_rate = self.config.params_learning_rate),loss='mean_squared_error')
        self.model.fit(
            X_train, y_train, 
            validation_data=(X_val, y_val),
            epochs=self.config.params_epochs,
            batch_size=self.config.params_batch_size
        )
        self.save_model(
                path=self.config.trained_model_path,
                model=self.model
            )

In [7]:
try:
    config = ConfigurationManager()
    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_base_model()
    training.train()
    
except Exception as e:
    raise e

[2024-05-21 17:25:35,070: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-05-21 17:25:35,084: INFO: common: yaml file: params.yaml loaded successfully]
[2024-05-21 17:25:35,085: INFO: common: created directory at: artifacts]
[2024-05-21 17:25:35,087: INFO: common: created directory at: artifacts\training]
[2024-05-21 17:25:35,333: WARNING: legacy_h5_format: No training configuration found in the save file, so the model was *not* compiled. Compile it manually.]
250/250 ━━━━━━━━━━━━━━━━━━━━ 44s 171ms/step - loss: 104.5612 - val_loss: 4.4275
[2024-05-21 17:26:19,374: WARNING: saving_api: You are saving your model as an HDF5 file via `model.save()` or `keras.saving.save_model(model)`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')` or `keras.saving.save_model(model, 'my_model.keras')`. ]
